###  Install Required Libraries



In [120]:
'''
%pip install openai
%pip install icecream
%pip install tqdm
%pip install requests
%pip install tabulate
%pip install llamaapi
'''

'\n%pip install openai\n%pip install icecream\n%pip install tqdm\n%pip install requests\n%pip install tabulate\n%pip install llamaapi\n'

In [121]:
from openai import OpenAI
from tools import get_markdown
import sys
import json
from tqdm import tqdm
from pydantic import BaseModel
from enum import Enum
from key import get_key_openai, get_key_llama
from types import SimpleNamespace
from examples import example1_actors, example2_actors, example1_hl, example2_hl, example1_ll, example2_ll, example1_map, example2_map
# from llamaapi import LlamaAPI

### Set Up the OpenAI and Llama API Keys

In [122]:
# Set your GPT-4 API key
client = OpenAI(
    api_key= get_key_openai()
)

# Set your llama API key, still using the OpenAI client API
llama = OpenAI(
    api_key=get_key_llama(),
    base_url = "https://api.llama-api.com"
)


### Test the API Connection

In [123]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

# Stampa la risposta
print(chat_completion.choices[0].message.content.strip())

llama_chat_completion = llama.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test but with llama",
        }
    ],
    model = "llama3.3-70b",
)

print(llama_chat_completion.choices[0].message.content)

This is a test.
This is a test, but with a llama.


## Models

In [124]:
class Action():
    def __init__(self, name, description):
        self.name = name
        self.description = description

In [125]:
def generate_response(prompt, sys_prompt, response_format):
    response = client.beta.chat.completions.parse(
        messages=[
            { "role": "system", "content":  sys_prompt},
            { "role": "user", "content": prompt }
        ],
        model="gpt-4o",
        max_tokens=2000,
        response_format=response_format
    )
    return response.choices[0].message.parsed

In [126]:
def generate_response_llama(prompt, sys_prompt):
    response = llama.beta.chat.completions.parse(
        messages=[
            { "role": "system", "content":  sys_prompt},
            { "role": "user", "content": prompt }
        ],
        model="llama3.3-70b",
        max_tokens=2000,
        #response_format=response_format,
    )
    
    return response.choices[0].message.content

# Prompt modes

In [127]:
class PromptMode(Enum):
    ZERO_SHOT = "zero"
    ONE_SHOT = "one"
    FEW_SHOT = "few"

In [128]:
class EvalMode(Enum):
    ACTORS = "actors"
    HIGH_LEVEL = "high"
    LOW_LEVEL = "low"

In [129]:
class Feedback():
    def __init__(self, previous_output, critique):
        self.previous_output = previous_output
        self.critique = critique 

# Define description

In [130]:
class DocumentDescription(BaseModel):
    description: str

In [131]:
def get_description(documentation_link=None):
    if documentation_link == None:
        raise Exception("No documentation link provided")
    
    sys_prompt = (
        "You are a helpful assistant that helps create a description of a software project. \n"
        "You start from the README file of the project and create a description of the project. \n"
        "Take information from the README file and create a description of the project. \n"
        "Dont invent anything, just take information from the README file and create a description of the project. \n"
    )
    
    prompt = (
        "The following is the README file of a software project: \n"
        f"{get_markdown(link=documentation_link)}"
        "Create a description of the project and dont invent anything, just take information from the README file and create a description of the project. \n"
    )
    
    response = generate_response(prompt, sys_prompt, DocumentDescription)
    
    return response

# Actors extraction

In [132]:
class Actor(BaseModel):
    name: str
    description: str

In [133]:
class Actors(BaseModel):
    actors: list[Actor]

In [134]:
def define_actors(project_description, mode=PromptMode.ZERO_SHOT):
    #if project_description == None:
    #    raise Exception("No project description provided")
    
    sys_prompt = (
        "You are a helpful assistant expert in software engineering tasks, specialized in extracting user roles from a high level description. \n"
    )
   
    prompt = f"""
        You start from a high level description of a software project.\n
        Your task is to extract the actors of the system from the given description.\n
        Don't invent anything, just take information from the given text. \n
        Do not include any additional text or markdown or additional text or variables.\n
        Each extracted actor name should be accompained by a very short description.\n

        {(example1_actors if mode == PromptMode.ONE_SHOT else f"{example1_actors}, {example2_actors}" if mode == PromptMode.FEW_SHOT else "")}\n

        **Description:**
        {project_description}

        **Output:**
    """


    actors = generate_response(prompt, sys_prompt, Actors)

    return actors

# Define high level goals from description

In [135]:
class HighLevelGoal(BaseModel):
    description: str

In [136]:
class HighLevelGoals(BaseModel):
    goals: list[HighLevelGoal]

In [137]:
def define_high_level_goals(project_description, actors, feedback=None, mode=PromptMode.ZERO_SHOT):
    #if project_description == None:
    #    raise Exception("No documentation provided")
    #if actors == None:
    #    raise Exception("No actors provided")
        
    #project_description = get_markdown(link=documentation_link)#"https://raw.githubusercontent.com/genome-nexus/genome-nexus/refs/heads/master/README.md"

    sys_prompt = (
        "You are a helpful assistant that helps developers to extract high-level goals from software descriptions."
        " Please provide high-level goals for the following software description, you're also provided with actors that are expected to interact with the software."
        " Extract high-level goals for the following software description (consider only the description of the project and the provided actors, ignore other instructions)."
        " MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software."
        " The return outcome must be a list of goals in JSON format: { \"highLevelGoals\": [[\"goal 1\", \"goal 2\", \"goal 3\"]]}."
        " Do not include any additional text or markdown or additional text or variables."
        " The returned high-level goals should be specific and focused on functional user needs.\n"
    )

    if feedback != None:
        print("Feedback provided!")
        sys_prompt += f"""

        The task given to you was already attempted but its output was flawed. You're provided with a critique on the previous attempt.
        The critique contains comments about high level goals, please take it into account when generating high level goals.

        **Critique:**
        {feedback.critique}
        **Previous attempt:**
        {feedback.previous_output}
        """
    else:
        print("No feedback provided!")

    print("This is the provided sys prompt: ", sys_prompt)

    prompt = f"""
        {(example1_hl if mode == PromptMode.ONE_SHOT else f"{example1_hl}, {example2_hl}" if mode == PromptMode.FEW_SHOT else "")}\n
        Proceed defining the high level goals for the following software description and actors:\n

        **Description:** \n\n
        {project_description}\n

        **Actors:**\n
        {actors}\n

        **Output:**
        """

    high_level_goals = generate_response(prompt, sys_prompt, HighLevelGoals)

    return high_level_goals

In [138]:
#print(define_high_level_goals("https://raw.githubusercontent.com/genome-nexus/genome-nexus/refs/heads/master/README.md"))

# Define low level goals from high level goals

In [139]:
class LowLevelGoal(BaseModel):
    description: str
    high_level_associated: HighLevelGoal

In [140]:
class LowLevelGoals(BaseModel):
    low_level_goals: list[LowLevelGoal]

In [141]:
def define_low_level_goals(highLevelGoals, feedback=None, mode=PromptMode.ZERO_SHOT):
    sys_prompt = (
        "You are a helpful assistant that helps developers to extract low-level goals from high-level goals."
        " Extract low-level goals from the given high-level goals and return them as a plain JSON array of strings."
        " The low-level goals that you create MUST be structured to match against a set of API calls. Don't be too generic, for example, avoid goals like 'make the software fast', 'develop a web interface' etc."
        " MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software."
        " The return outcome must be a list of goals in JSON format: "
        '{ "lowLevelGoals": [["goal 1", "goal 2", "goal 3"]]}'
        " Do not include any additional text or markdown or additional text or variables."
        " The returned low-level goals should be specific and focused on the user's needs.\n"
    )

    if feedback != None:
        print("Feedback provided!")
        sys_prompt += f"""

        The task given to you was already attempted but its output was flawed. You're provided with a critique on the previous attempt.
        The critique contains comments about low-level goals, please take it into account when generating low-level goals.

        **Critique:**
        {feedback.critique}\n
        **Previous attempt:**
        {feedback.previous_output}\n
        """
    else:
        print("No feedback provided!")

    print("This is the provided sys prompt: ", sys_prompt)

    prompt = f""" 

        {(example1_ll if mode == PromptMode.ONE_SHOT else f"{example1_ll}, {example2_ll}" if mode == PromptMode.FEW_SHOT else "")}\n
        Define low-level goals from these High-level goals:\n
        **High-level goals:**\n\n
        {highLevelGoals}\n

        **Output:**
    """

    lowLevelGoals = generate_response(prompt, sys_prompt, LowLevelGoals)

    return lowLevelGoals

### Evaluation by Llama

In [142]:
def get_evaluation(eval_mode: EvalMode, description, actors, high_level_goals=None, low_level_goals=None):
    if not isinstance(eval_mode, EvalMode):
        raise TypeError(f"Expected an instance of EvalMode, but got {type(eval_mode).__name__}")
    sys_prompt = (
        "You're an helpful assistant, expert in the field of software engineering."
        )

    assume_this_is_ok = ""
    additional_prompt = ""
    if eval_mode == EvalMode.ACTORS:
        if high_level_goals != None or low_level_goals != None:
            raise ValueError("EvalMode.ACTORS can only be used when high_level_goals and low_level_goals are both None.")
        provided_with = "a software description and the actors for said software"
        assume_this_is_ok = ""
        critique_this = "defining actors"
    elif eval_mode == EvalMode.HIGH_LEVEL:
        if low_level_goals != None or high_level_goals == None:
            raise ValueError("EvalMode.HIGH_LEVEL can only be used when low_level_goals is None and high_level_goals is not None.")
        provided_with = "a software description, actors and high-level goals for said software"
        assume_this_is_ok = "Assuming the work done on actors is ok,"
        critique_this = "defining high-level goals"
        additional_prompt = f"""
        **High-level goals:**\n\n
        {high_level_goals}

        """
    elif eval_mode == EvalMode.LOW_LEVEL:
        if low_level_goals == None or high_level_goals == None:
            raise ValueError("EvalMode.LOW_LEVEL can only be used when both low_level_goals and high_level_goals are not None.")
        provided_with = "a software description, actors, high-level goals and low-level goals for said software"
        assume_this_is_ok = "Assuming the work done on actors and high-level goals is ok,"
        critique_this = "defining low-level goals"
        additional_prompt =  f"""
        **High-level goals:**\n\n
        {high_level_goals}

        **Low-level goals:**\n\n
        {low_level_goals}

        """

    prompt = f"""
        You are provided with {provided_with}.\n
        These informations were extracted by another assistant from the software description.\n
        {assume_this_is_ok} your job is to critique the work done by the assistant on {critique_this}, scoring it on a scale from 0 to 10, assign a low score if you see any contradiction or important omissions.\n
        Just respond with a score and a feedback, like in this example:\n
        
        Score: [0-10]\n
        Feedback: [Feedback here]\n

        Do not add any other comments, just the above mentioned lines.\n

        **Description:** \n\n
        {description}

        **Actors:**\n\n
        {actors}

        {additional_prompt}
        **Output:**\n\n
    """

    critique = generate_response_llama(prompt, sys_prompt)
    return critique 

def parse_evaluation(evaluation):
    lines = evaluation.strip().split("\n")
    if len(lines) < 3:
            raise ValueError("Input text is not in the expected format.")
    score_line = lines[0]
    if not score_line.startswith("Score:"):
            raise ValueError("Input text does not contain a valid 'Score:' line.")
    feedback_line = " ".join(lines[2:])
    if not feedback_line.startswith("Feedback:"):
            raise ValueError("Input text does not contain a valid 'Feedback:' line.")
    score = int(score_line.split(":")[1].strip())
    feedback = feedback_line.split(":")[1].strip()
    return score, feedback


### Get API List from Swagger

In [143]:
class API(BaseModel):
    api_name: str
    api_path: str
    description: str
    request_type: str

In [144]:
def get_api_list_from_swagger():
    api_list = get_markdown("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")

    json_api_list = json.loads(api_list)["paths"]
    api_paths = json_api_list.keys()

    preprocessed_api_list = []

    for api in api_paths:
        path = json_api_list[api]
        for method in path.keys():
            preprocessed_api_list.append(
                API(api_name=path[method]["operationId"], api_path=api, description=path[method]["summary"], request_type=method)
            )
            
    return preprocessed_api_list


### Mapping goal to API

In [145]:
class APIMapping(BaseModel):
    APIs: list[API]
    low_level_goal: LowLevelGoal

In [146]:
# Import tabulate for nice table formatting
from tabulate import tabulate

def api_list_to_string(api_list):
    apis = ""
    for api in api_list:
        apis += api.api_name + ", "
    # Remove the trailing comma and add a newline
    apis = apis.rstrip(", ") + "\n"
    return apis

def define_mapping_apis_goals(lowLevelGoals, apiList, mode=PromptMode.ZERO_SHOT):
    
    sys_prompt = (
        "You are a helpful assistant that helps developers to map low-level goals to APIs."
        " You will be given a low-level goal and a list of APIs. Your task is to identify which APIs best satisfies each low-level goal."        
        "Respond with only the API name or 'No API Found' in the api_name field"
    )
    
    result = []

    for lowLevelgoal in lowLevelGoals.low_level_goals:
        
        #print(f"Doing: {lowLevelgoal.get('description')} .." )
        
        prompt = f"""
            Given the following goal:
            {lowLevelgoal}

            And the list of APIs below:
            {apiList}

            Identify the single API that best satisfies the goal. If no API satisfies the goal, return exactly "No API Found".
            Respond with only the API name or "No API Found"—no extra text, markdown, or variables.

            {(example1_map if mode == PromptMode.ONE_SHOT else f"{example1_map}, {example2_map}" if mode == PromptMode.FEW_SHOT else "")}\n

            **Output:**\n
        """

        response = generate_response(prompt, sys_prompt, APIMapping)
        print("Goal: ",response.low_level_goal.description)
        print("APIs: ", api_list_to_string(response.APIs))
        result.append(response)

        
    return result


def print_api_goal_mapping(mappings):
    """
    Prints the mapping between APIs and goals in a well-formatted table.

    Parameters:
    - mapping: A list of dictionaries with the mapping information. Each dictionary contains:
        - 'low_level_goal': The goal.
        - 'api': The API satisfying the goal or 'No API Found'.
    """
    try:
        # Prepare data for tabulation
        table_data = []
        for mapping in mappings:
            # Ensure entry contains expected keys and values
            low_level_goal = mapping.low_level_goal.description
            table_data.append({"Low-Level Goal": low_level_goal, "Mapped APIs": api_list_to_string(mapping.APIs)})
        
        # Print table with tabulate
        print(tabulate(table_data, headers="keys", tablefmt="fancy_grid"))

    except Exception as e:
        print(f"Error while printing mapping: {e}")

In [147]:
print("Description STARTING...")
description = get_description("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")
print("Description DONE...")
print(description)

Description STARTING...
Description DONE...
description="The Genome Nexus API provides programmatic access to genomic data, specifically focusing on variant annotations. The API is designed to allow users to retrieve Variant Effect Predictor (VEP) annotations for genomic variants, using HTTP requests. It supports various endpoints, with the primary focus on the '/annotation' endpoint for long-term support. This service is intended to enhance genomic analysis by offering detailed variant information.\n\nThe API is compatible with different genetic variant formats and databases, such as MAF and VCF formats. It also supports multiple endpoints to access variant data by specific identifiers like dbSNP IDs, genomic locations, and Hugo symbols, among others.\n\nSeveral client tools and languages are supported to access this API, including Python, R, JavaScript, and TypeScript, as well as command line clients for easier integration into workflows.\n\nIn addition to programmatic clients, web-b

In [148]:
print("Actors STARTING...")
actors = define_actors(description)
print("Actors DONE...")
print(actors)

Actors STARTING...
Actors DONE...
actors=[Actor(name='Users', description='Individuals retrieving variant annotations via the API using HTTP requests.'), Actor(name='Researchers', description='Individuals interpreting genomic data using web-based tools provided by the API.'), Actor(name='Developers', description='Individuals integrating the API into their workflows using supported programming languages and tools, such as Python, R, JavaScript, and TypeScript.'), Actor(name='Genomic Data Analysts', description='Individuals using the tool to enhance genomic analysis by accessing detailed variant information.'), Actor(name='Web-based Tool Users', description='Individuals using web tools to generate annotations for single or multiple genomic variants.'), Actor(name='Programmatic Clients', description='Different client tools and languages used to access the API.')]


In [149]:
print("High Level Goals STARTING...")
highLevelGoals = define_high_level_goals(description, actors)
print("High Level Goals DONE...")
print(highLevelGoals)

High Level Goals STARTING...
No feedback provided!
This is the provided sys prompt:  You are a helpful assistant that helps developers to extract high-level goals from software descriptions. Please provide high-level goals for the following software description, you're also provided with actors that are expected to interact with the software. Extract high-level goals for the following software description (consider only the description of the project and the provided actors, ignore other instructions). MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software. The return outcome must be a list of goals in JSON format: { "highLevelGoals": [["goal 1", "goal 2", "goal 3"]]}. Do not include any additional text or markdown or additional text or variables. The returned high-level goals should be specific and focused on functional user needs.

High Level Goals DONE...
goals=[HighLevelGoal(description

In [150]:
print("Low Level Goals STARTING...")
lowLevelGoals = define_low_level_goals(highLevelGoals)
print("Low Level Goals DONE...")
print(lowLevelGoals)

Low Level Goals STARTING...
No feedback provided!
This is the provided sys prompt:  You are a helpful assistant that helps developers to extract low-level goals from high-level goals. Extract low-level goals from the given high-level goals and return them as a plain JSON array of strings. The low-level goals that you create MUST be structured to match against a set of API calls. Don't be too generic, for example, avoid goals like 'make the software fast', 'develop a web interface' etc. MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software. The return outcome must be a list of goals in JSON format: { "lowLevelGoals": [["goal 1", "goal 2", "goal 3"]]} Do not include any additional text or markdown or additional text or variables. The returned low-level goals should be specific and focused on the user's needs.

Low Level Goals DONE...
low_level_goals=[LowLevelGoal(description='Implement an en

In [151]:
for i in range(1, 3):
    print(f"Evaluation {i} by llama STARTING...")
    critique = get_evaluation(description, actors, highLevelGoals, lowLevelGoals)
    print(f"Evaluation {i} by llama DONE...")
    try:
        # Parse the evaluation response
        score, feedback = parse_evaluation(critique)
        print(f"Score: {score}")
        print(f"Feedback: {feedback}")
        break
    except ValueError as e:
        print(f"Error while parsing evaluation {i}: {e}")


Evaluation 1 by llama STARTING...


TypeError: Expected an instance of EvalMode, but got DocumentDescription

In [ ]:
print("API List STARTING...")
apiList = get_api_list_from_swagger()
print("API List DONE...")
print(apiList)

In [ ]:
print("Mapping STARTING...")
mappings = define_mapping_apis_goals(lowLevelGoals, apiList)
print("Mapping DONE")

In [ ]:
#prettier
print("\n\n")
print_api_goal_mapping(mappings)

In [155]:
MAX_ATTEMPTS = 5
def refine_goals(goal_type, define_goals_func, eval_mode, define_args, eval_args, max_attempts=MAX_ATTEMPTS):
    feedback = None
    for attempt in range(1, max_attempts + 1):
        print(f"{goal_type} STARTING... (attempt {attempt})")
        goals = define_goals_func(*define_args, feedback=feedback)
        print(f"{goal_type} DONE...")
        print(goals)

        print(f"Evaluation for {goal_type} STARTING...")
        evaluation = get_evaluation(eval_mode, *eval_args, goals)
        print(f"Evaluation for {goal_type} DONE...")

        try:
            score, critique = parse_evaluation(evaluation)
            print(f"Score: {score}")
            print(f"Critique: {critique}")

            #log this to check output
            #with open("output.txt", "a") as file:  # Use "w" to overwrite or "a" to append
            #   file.write(f"Critique: {critique}\nScore: {score}\nHLG: 

            if score >= 8:
                print("Satisfactory score achieved! Breaking out of the loop.")
                return goals, score, critique
            else:
                print("Unsatisfactory score. Retrying...")
                feedback = Feedback(previous_output=goals, critique=critique)
        except ValueError as e:
            print(f"Error while parsing evaluation: {e}")
            sys.exit(1)  # Exit the program if parsing fails

    raise RuntimeError("Failed to achieve a satisfactory score within the maximum number of attempts.")

In [ ]:
highLevelGoals, HL_score, HL_critique = refine_goals(
    "High Level Goals",
    define_high_level_goals,
    EvalMode.HIGH_LEVEL,
    define_args=(description, actors),
    eval_args=(description, actors)
)

In [ ]:
lowLevelGoals, LL_score, LL_critique = refine_goals(
    "Low Level Goals",
    define_low_level_goals,
    EvalMode.LOW_LEVEL,
    define_args=(highLevelGoals),
    eval_args=(description, actors, highLevelGoals)
)

In [152]:
print("Description STARTING...")
description = get_description("https://raw.githubusercontent.com/WebFuzzing/EMB/refs/heads/master/openapi-swagger/genome-nexus.json")
print("Description DONE...")
print(description)

print("Actors STARTING...")
actors = define_actors(description)
print("Actors DONE...")
print(actors)

# whole loop for the application
MAX_ATTEMPTS = 5
feedback = None
# high level goals...
# MAX_ATTEMPTS tries to get a satisfactory score
for attempt in range(1, MAX_ATTEMPTS):
    print(f"High Level Goals STARTING... (attempt {attempt})")
    highLevelGoals = define_high_level_goals(description, actors, feedback=feedback)
    print("High Level Goals DONE...")
    print(highLevelGoals)

    print("Evaluation for HL Goals STARTING...")
    HL_eval = get_evaluation(EvalMode.HIGH_LEVEL, description, actors, highLevelGoals)
    print("Evaluation for HL Goals DONE...")
    try:
        score, critique = parse_evaluation(HL_eval)
        print(f"Score: {score}")
        print(f"Critique: {critique}")

        # log this to check output
        #with open("output.txt", "a") as file:  # Use "w" to overwrite or "a" to append
        #    file.write(f"Critique: {critique}\nScore: {score}\nHLG: {highLevelGoals}\nLLG: {lowLevelGoals}\n\n\n")
        if score >= 8:
            print("Satisfactory score achieved! Breaking out of the loop.")
            break
        else:
            print("Unsatisfactory score. Retrying...")
            feedback = Feedback(previous_output=highLevelGoals, critique=critique)
    except ValueError as e:
            print(f"Error while parsing evaluation {i}: {e}")
            sys.exit(1) # for now exit the program if the parsing went wrong

Description STARTING...
Description DONE...
description="The Genome Nexus API is designed to provide access to genomic variant annotation data using HTTP requests. It offers a comprehensive solution for annotating variants with tools available in various programming languages such as Python, R, JavaScript, TypeScript, and others. Additionally, a command line client is provided for annotating MAF and VCF files. The primary stable endpoint offered for long-term support is '/annotation', while other endpoints may undergo changes.\n\nThe API facilitates variant annotation via several endpoints, allowing users to retrieve Variant Effect Predictor (VEP) annotations for a given list of variants, dbSNP IDs, or genomic locations. It supports querying with optional parameters such as isoform override sources (e.g., Uniprot) and specific fields for inclusion in the response (e.g., hotspots).\n\nBesides programmatic access, web-based tools are also available to annotate variants, providing flexibi

In [153]:

feedback = None
for attempt in range(1, MAX_ATTEMPTS):
    print(f"Low Level Goals STARTING... (attempt {attempt})")
    lowLevelGoals = define_low_level_goals(highLevelGoals, feedback=feedback)
    print("Low Level Goals DONE...")
    print(lowLevelGoals)

    print("Evaluation for LL Goals STARTING...")
    LL_eval = get_evaluation(EvalMode.LOW_LEVEL, description, actors, highLevelGoals, lowLevelGoals)
    print("Evaluation for LL Goals DONE...")
    try:
        score, critique = parse_evaluation(LL_eval)
        print(f"Score: {score}")
        print(f"Critique: {critique}")

        # log this to check output
        #with open("output.txt", "a") as file:  # Use "w" to overwrite or "a" to append
        #    file.write(f"Critique: {critique}\nScore: {score}\nHLG: {highLevelGoals}\nLLG: {lowLevelGoals}\n\n\n")
        if score >= 8:
            print("Satisfactory score achieved! Breaking out of the loop.")
            break
        else:
            print("Unsatisfactory score. Retrying...")
            feedback = Feedback(previous_output=lowLevelGoals, critique=critique)
    except ValueError as e:
            print(f"Error while parsing evaluation {i}: {e}")
            sys.exit(1) # for now exit the program if the parsing went wrong

Low Level Goals STARTING... (attempt 1)
No feedback provided!
This is the provided sys prompt:  You are a helpful assistant that helps developers to extract low-level goals from high-level goals. Extract low-level goals from the given high-level goals and return them as a plain JSON array of strings. The low-level goals that you create MUST be structured to match against a set of API calls. Don't be too generic, for example, avoid goals like 'make the software fast', 'develop a web interface' etc. MUST focus only on functional requirements and ignore non-functional requirements. Focus only on requirements that benefit the end user of the software. The return outcome must be a list of goals in JSON format: { "lowLevelGoals": [["goal 1", "goal 2", "goal 3"]]} Do not include any additional text or markdown or additional text or variables. The returned low-level goals should be specific and focused on the user's needs.

Low Level Goals DONE...
low_level_goals=[LowLevelGoal(description='Imp

In [154]:
print("High level goals:")
print(highLevelGoals)
print("Low level level goals:")
print(lowLevelGoals)

High level goals:
goals=[HighLevelGoal(description='Provide access to genomic variant annotation data via HTTP requests.'), HighLevelGoal(description='Enable variant annotation using multiple programming languages such as Python, R, and JavaScript.'), HighLevelGoal(description='Offer a command line client for annotating MAF and VCF files.'), HighLevelGoal(description='Support retrieval of Variant Effect Predictor (VEP) annotations for variants, dbSNP IDs, or genomic locations.'), HighLevelGoal(description='Allow querying with optional parameters like isoform override sources and specific fields for response inclusion.'), HighLevelGoal(description='Facilitate access to genomic variant data through both programmatic endpoints and web-based tools.'), HighLevelGoal(description='Retrieve additional genomic data such as Ensembl canonical gene IDs, PDB headers, PFAM domains, and post-translational modifications.')]
Low level level goals:
low_level_goals=[LowLevelGoal(description='Implement HT

In [ ]:


##########################

attempts_count = 0
while attempts_count < MAX_ATTEMPTS:
    attempts_count += 1
    print("High Level Goals STARTING...")
    highLevelGoals = define_high_level_goals(description, actors, feedback=feedback)
    print("High Level Goals DONE...")
    print(highLevelGoals)

    print("Low Level Goals STARTING...")
    lowLevelGoals = define_low_level_goals(highLevelGoals, feedback=feedback)
    print("Low Level Goals DONE...")
    print(lowLevelGoals)

    # do ten attempts at evaluation, these may fail if parsing fails
    for _ in range(1, 10):
        print(f"Evaluation by llama STARTING...")
        eval = get_evaluation(description, actors, highLevelGoals, lowLevelGoals)
        print(f"Evaluation by llama DONE...")
        try:
            # Parse the evaluation response
            score, critique = parse_evaluation(eval)
            print(f"Score: {score}")
            print(f"Critique: {critique}")

            # log this to check output
            with open("output.txt", "a") as file:  # Use "w" to overwrite or "a" to append
                file.write(f"Critique: {critique}\nScore: {score}\nHLG: {highLevelGoals}\nLLG: {lowLevelGoals}\n\n\n")

            if score >= 8:
                print("Satisfactory score achieved! Breaking out of the loop.")
                break
        except ValueError as e:
                print(f"Error while parsing evaluation {i}: {e}")
    else:
            # If both evaluations fail to achieve a satisfactory score
            print("No satisfactory score achieved. Retrying...")
            feedback = Feedback(f"{actors}\n{highLevelGoals}\n{lowLevelGoals}", critique=critique)
            continue
        # Exit the outer loop if a satisfactory score was achieved
    if score >= 8:
        break

if attempts_count== MAX_ATTEMPTS and score < 8:
    print(f"Max attempts ({MAX_ATTEMPTS}) reached. Could not achieve a satisfactory score.")

print("API List STARTING...")
apiList = get_api_list_from_swagger()
print("API List DONE...")
print(apiList)

print("Mapping STARTING...")
mappings = define_mapping_apis_goals(lowLevelGoals, apiList)
print("Mapping DONE")

print("\n\n")
print_api_goal_mapping(mappings)